# 2x2 Test classifier
## Mingde Yin, March 7 2021

This implements a 128 layer convolutional neural network to determine the factorizability of bipartite states.

In [ ]:
import tensorflow as tf
import numpy as np

Make a model which accepts a 4x4 matrix with real and imaginary components

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (4,4,2)), 
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               4224      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 4,353
Trainable params: 4,353
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

## Matrix generator and validator functions from previously

In [ ]:
def random_pure_density_matrix(N):
    '''
    Makes a random pure density matrix for an N level system.
    This is done by taking the state vectors for a pure state.

    Obeys:
    - Hermiticity
    - Tr = 1
    - P^2 = P
    '''
    # state vector
    reals = np.random.random(N)
    imgs = np.random.random(N) * 1j
    state = np.matrix((reals+imgs).reshape((N, 1)))

    # normalize
    state = state / np.linalg.norm(state)

    # make density matrix
    return np.matmul(state, state.H)

def bipartite_state(A, B):
    '''
    Returns full state of two density matrices representing factorized states
    Tensor product implemented using kronecker product of the matrices
    '''
    return np.matrix(np.kron(A, B))

def partial_transpose(P):
    '''
    partial transposes matrix in place
    '''
    # Partial transpose the array
    for x in range(1, P.shape[0], 2):
        for y in range(1, P.shape[1], 2):
            P[x-1:x+1, y-1:y+1] = P[x-1:x+1, y-1:y+1].T
    return P


def entangled_PPT(P):
    '''
    determines if a state is entangled by the Horodecki criterion

    only works for 2 (x) 2 and 2 (x) 3 cases.
    '''
    Ptb = np.copy(P)

    partial_transpose(Ptb)

    evs = np.linalg.eigvals(Ptb)
    return np.all(evs.real > 0)


## Generate 1000 matrices to test

In [ ]:
np.random.seed(1)

matrices = np.zeros((10000, 4, 4, 2))
results = np.zeros(10000)

for i in range(10000):
  mat1 = random_pure_density_matrix(2)
  mat2 = random_pure_density_matrix(2)
  mat = bipartite_state(mat1, mat2)

  matrices[i,:,:,0] = mat.real
  matrices[i,:,:,1] = mat.imag
  results[i] = entangled_PPT(mat)


# Train the network

In [ ]:
history = model.fit(x=matrices, y=results,
      steps_per_epoch=100,  
      epochs=100,
      verbose=1)

Epoch 1/100
100/100 [==============================] - 0s 1ms/step - loss: 0.4875 - accuracy: 0.8868
Epoch 2/100
100/100 [==============================] - 0s 2ms/step - loss: 0.3410 - accuracy: 0.8943
Epoch 3/100
100/100 [==============================] - 0s 2ms/step - loss: 0.3339 - accuracy: 0.8971
Epoch 4/100
100/100 [==============================] - 0s 2ms/step - loss: 0.3353 - accuracy: 0.8961
Epoch 5/100
100/100 [==============================] - 0s 1ms/step - loss: 0.3336 - accuracy: 0.8963
Epoch 6/100
100/100 [==============================] - 0s 1ms/step - loss: 0.3382 - accuracy: 0.8943
Epoch 7/100
100/100 [==============================] - 0s 2ms/step - loss: 0.3301 - accuracy: 0.8974
Epoch 8/100
100/100 [==============================] - 0s 2ms/step - loss: 0.3314 - accuracy: 0.8964
Epoch 9/100
100/100 [==============================] - 0s 2ms/step - loss: 0.3270 - accuracy: 0.8992
Epoch 10/100
100/100 [==============================] - 0s 2ms/step - loss: 0.3414 - accura

# Try to predict

This is literally garbage right now

1 = Factorizable
0 = Entangled

Neural network outputs something between 0 and 1

In [ ]:
matrices = np.zeros((20, 4, 4, 2))
results = np.zeros(20)

for i in range(20):
  mat1 = random_pure_density_matrix(2)
  mat2 = random_pure_density_matrix(2)
  mat = bipartite_state(mat1, mat2)

  matrices[i,:,:,0] = mat.real
  matrices[i,:,:,1] = mat.imag
  results[i] = entangled_PPT(mat)

print(results)
print(model.predict(x=matrices, batch_size=20))



[0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
[[0.11785063]
 [0.10505965]
 [0.12163958]
 [0.11472026]
 [0.09796125]
 [0.10600638]
 [0.09864578]
 [0.07458401]
 [0.10841855]
 [0.12761122]
 [0.15103939]
 [0.1196062 ]
 [0.07676035]
 [0.10476428]
 [0.11031389]
 [0.17496276]
 [0.14809957]
 [0.08922359]
 [0.11569563]
 [0.1318663 ]]


# Reflection

1. Need a proper way of figuring out whether or not a state is entangled or not
2. Need to find a better network architecture

The problem with convolution kernels is that they really like to compress and scrunch up information -- which is OK for images, but maybe not for matrices with precise numbers in them